In [ ]:
!pip install torch_geometric -q

In [ ]:
from torch_geometric.datasets import *
import torch_geometric.transforms as T

import time

import torch
import torch.nn.functional as F
from torch import tensor
from torch.optim import Adam
import numpy as np
from torch_geometric.utils import *
import networkx as nx
from tqdm import tqdm

In [ ]:
url = f"https://data.pyg.org/whl/torch-{torch.__version__}.html"
!pip install torch-scatter -f {url}

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing, GCNConv
from torch_scatter import scatter_add
from torch_geometric.utils import add_remaining_self_loops
import torch.nn as nn

In [ ]:
Linear(10,2).reset_parameters()

In [ ]:
def gcn_norm(edge_index, edge_weight=None, num_nodes=None, improved=False,
             add_self_loops=True, dtype=None):

    fill_value = 2. if improved else 1.
    num_nodes = int(edge_index.max()) + 1 if num_nodes is None else num_nodes
    if edge_weight is None:
        edge_weight = torch.ones((edge_index.size(1), ), dtype=dtype,
                                 device=edge_index.device)

    if add_self_loops:
        edge_index, tmp_edge_weight = add_remaining_self_loops(
            edge_index, edge_weight, fill_value, num_nodes)
        assert tmp_edge_weight is not None
        edge_weight = tmp_edge_weight

    row, col = edge_index[0], edge_index[1]
    deg = scatter_add(edge_weight, col, dim=0, dim_size=num_nodes)
    deg_inv_sqrt = deg.pow_(-0.5)
    deg_inv_sqrt.masked_fill_(deg_inv_sqrt == float('inf'), 0)
    return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

class Prop(MessagePassing):
    def __init__(self, num_classes, K, bias=True, **kwargs):
        super(Prop, self).__init__(aggr='add', **kwargs)
        self.K = K
        self.proj = Linear(num_classes, 1)

    def forward(self, x, edge_index, edge_weight=None):
        # edge_index, norm = GCNConv.norm(edge_index, x.size(0), edge_weight, dtype=x.dtype)
        edge_index, norm = gcn_norm(edge_index, edge_weight, x.size(0), dtype=x.dtype)


        preds = []
        preds.append(x)
        for k in range(self.K):
            x = self.propagate(edge_index, x=x, norm=norm)
            preds.append(x)

        pps = torch.stack(preds, dim=1)
        retain_score = self.proj(pps)
        retain_score = retain_score.squeeze()
        retain_score = torch.sigmoid(retain_score)
        retain_score = retain_score.unsqueeze(1)
        out = torch.matmul(retain_score, pps).squeeze()
        return out

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j

    def __repr__(self):
        return '{}(K={})'.format(self.__class__.__name__, self.K)


class Net(torch.nn.Module):
    def __init__(self, ni,nf,dp=0.5):
        super(Net, self).__init__()

        # linear layer
        self.lin = nn.Sequential(nn.Dropout(dp),
                                  Linear(ni, ni//4),
                                  nn.ReLU(),
                                  nn.Dropout(dp),
                                  Linear(ni//4, nf) )

        # MPNN
        self.prop = Prop(nf, 10)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.lin(x)

        x = self.prop(x, edge_index)

        # x = F.log_softmax(x, dim=1) # for classification

        return x

In [ ]:
ni = dataset.num_features
nf = dataset.num_classes
ni,nf

(1433, 7)

In [ ]:
model = Net(ni,nf)

In [ ]:
data.x.shape

torch.Size([2708, 1433])

In [ ]:
data.edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [ ]:
model

Net(
  (lin): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=1433, out_features=358, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=358, out_features=7, bias=True)
  )
  (prop): Prop(K=10)
)

In [ ]:
out = model(data)

In [ ]:
out.shape

torch.Size([2708, 7])

In [ ]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
model

Net(
  (lin): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=1433, out_features=358, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=358, out_features=7, bias=True)
  )
  (prop): Prop(K=10)
)

In [ ]:
def get_data( normalize_features=False, transform=None):
    dataset = QM7b('.')

    if transform is not None and normalize_features:
        dataset.transform = T.Compose([T.NormalizeFeatures(), transform])
    elif normalize_features:
        dataset.transform = T.NormalizeFeatures()
    elif transform is not None:
        dataset.transform = transform

    return dataset

In [ ]:
dataset = get_data()

Processing...
Done!


In [ ]:
dataset = get_planetoid('cora')

In [ ]:
dataset[10]

tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
         4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5],
        [0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5,
         0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]])

In [ ]:
dataset.get_summary()

QM7b (#graphs=7211):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     15.4 |    245   |
| std        |      2.7 |     82.7 |
| min        |      4   |     16   |
| quantile25 |     14   |    196   |
| median     |     15   |    225   |
| quantile75 |     17   |    289   |
| max        |     23   |    529   |
+------------+----------+----------+

In [ ]:
data = dataset[0]

In [ ]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [ ]:
runs=5
pbar = tqdm(range(runs), unit='run')

  0%|          | 0/5 [00:00<?, ?run/s]

In [ ]:
for i in pbar:
  print('aa')

100%|██████████| 5/5 [00:20<00:00,  4.10s/run]

aa
aa
aa
aa
aa


In [ ]:
device='cpu'
data = data.to(device)